In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import gtfstk as gt
import shapely.geometry as sg
import shapely.ops as so
import folium as fl

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = Path('../data')

%load_ext autoreload
%autoreload 2


In [2]:
import shapely.wkt as sw
import shapely.ops as so

#s = sg.shape(pfeed.shapes['features'][0]['geometry'])
s = sw.loads("LINESTRING(0 0, 0.2 0.1, 0.3 0.6, 0.4 0.1, 1 1)")
#s = sg.LineString([(x, y) for x, y in reversed(s.coords)])

eps = 10e-3
b0 = s.buffer(eps, cap_style=3)
b1 = s.buffer(0.05, cap_style=2)
diff = b1.difference(b0)
left, right = list(so.polygonize(diff))


In [3]:
from matplotlib import pyplot
from shapely.geometry import LineString
from descartes.patch import PolygonPatch

fig = pyplot.figure(1, figsize=(10, 4), dpi=180)


ax = fig.add_subplot(121)
patch2a = PolygonPatch(left, facecolor='#cccccc', edgecolor='#999999')
ax.add_patch(patch2a)
patch2b = PolygonPatch(right, facecolor='#99ccff', edgecolor='#6699cc')
ax.add_patch(patch2b)

ax.set_xlim(-1, 2)
ax.set_ylim(-1, 2)

pyplot.show()

<Figure size 1800x720 with 1 Axes>

In [4]:
pfeed = mg.ProtoFeed(DATA_DIR/'auckland')
pfeed.shapes_extra

{'shp1': 2, 'shp2': 2}

In [5]:
# Visually test geometrize_stops()

# Compute
geo_stops = gt.geometrize_stops(pfeed.stops, use_utm=True)
linestring_by_shape = mg.build_geometry_by_shape(pfeed, use_utm=True)
linestring = linestring_by_shape['shp2']
stops_by_side = {
    side: mg.get_nearby_stops(geo_stops, linestring, side).to_crs(gt.WGS84)
    for side in ['left', 'right', 'both']
}

# Plot
collection_by_side = {side: json.loads(stops.to_json()) 
  for side, stops in stops_by_side.items()}
center = list(stops_by_side['left'].geometry.iat[0].coords[0])[::-1]
my_map = fl.Map(location=center, zoom_start=13, tiles='cartodbpositron')

# Path
f = pfeed.shapes['features'][1]
prop = f['properties']
prop['color'] = 'black'
path = fl.GeoJson(f,
    name=prop['shape_id'],
    style_function=lambda x: {
      'color': x['properties']['color']},
)
path.add_to(my_map)

# Stops
color_by_side = {
    'left': 'blue',
    'right': 'red',
    'both': 'gray',
}
radius_by_side = {
    'left': 5,
    'right': 5,
    'both': 8,
}
for side in stops_by_side:
    color = color_by_side[side]
    radius = radius_by_side[side]
    for f in collection_by_side[side]['features']:
        prop = f['properties']
        if f['geometry']['type'] == 'Point':
            lon, lat = f['geometry']['coordinates']
            fl.CircleMarker(
                location=[lat, lon],
                radius=radius,
                fill=False,
                color=color,
                weight=2,
            ).add_to(my_map)

my_map

In [6]:
feed = mg.build_feed_from_pfeed(pfeed)


  route_id                           trip_id  direction_id shape_id  \
0       rA  t-rA-weekday_peak_1-07:00:00-1-0             1   shp1-1   
1       rA  t-rA-weekday_peak_1-07:00:00-1-1             1   shp1-1   
2       rA  t-rA-weekday_peak_1-07:00:00-1-2             1   shp1-1   
3       rA  t-rA-weekday_peak_1-07:00:00-1-3             1   shp1-1   
4       rA  t-rA-weekday_peak_1-07:00:00-1-4             1   shp1-1   

   service_id service_window_id route_short_name  \
0  srv1111100    weekday_peak_1                A   
1  srv1111100    weekday_peak_1                A   
2  srv1111100    weekday_peak_1                A   
3  srv1111100    weekday_peak_1                A   
4  srv1111100    weekday_peak_1                A   

                                          route_desc  route_type  frequency  \
0  City Link, Wynyard Quarter to Karangahape Rd v...           3         12   
1  City Link, Wynyard Quarter to Karangahape Rd v...           3         12   
2  City Link, Wynyard Q

In [7]:
prefix = 't-rB-weekday_peak_1-07:00:00'
t = (
    feed.trips
    .loc[lambda x: x.trip_id.str.startswith(prefix)]
)
m = feed.map_trips([t.trip_id.iat[0], t.trip_id.iat[-1]])


In [8]:
m.location, my_map.location

([0, 0], [-36.84834, 174.74393])

In [9]:
gt.make_html?